In [ ]:
# category_list 확인하는 코드
import math 
from PIL import Image, ImageDraw, ImageFont
import re
import os
import shutil
import json
import xml.etree.ElementTree as ET
 
path = "./annotations_smartinside_segmentation/" # annotation(json)이 있는 경로
file_list = os.listdir(path) # 해당 경로의 모든 파일을 리스트로 받기
 
path3 = "./dataset_hair_new_smartinside_segmentation/" # 원본 jpg데이터가 있는 폴더 경로
path4 = "./new_bbox_smartinside_segmentation/" # 새로운 라벨링(txt)데이터가 저장되는 경로
file_list2 = os.listdir(path3) # 지정 폴더안 원본 jpg파일들을 리스트로

category_list = [] # 원하는 결과를 위한 빈 리스트 생성

for line in file_list2: # 모든 라벨링 파일을 한번 훑어봄
    filename = os.path.basename(line) # 파일 이름을 변수로 저장
    print(filename) # 파일 이름들 출력(생략가능)
    img = Image.open(path3 + str(os.path.splitext(filename)[0]) + '.jpg') # 같은 이름을 가진 path3경로에 있는 jpg 파일 열기
    
    null = None # null이 지정이 안되어있어서 따로 지정
    
    with open(path+str(os.path.splitext(filename)[0]) + '.json','r', encoding='utf-8-sig') as inf: # path경로에 있는 모든 json 파일을 열기
        obj = eval(inf.read())
    x = obj['annotations'] # json안에 있는 딕셔너리에서 annotation의 value값들을 x에 지정
    for i in x:
        if i['category'] in category_list: # category_list에 기존 category(object name)이 존재하면 넘김
            pass
        else:
            category_list.append(i['category']) # category_list에 새로운게 발견되면 추가

print(category_list)    

In [33]:
import contextlib
import json
from pathlib import Path
import cv2
import pandas as pd
from PIL import Image
from collections import defaultdict
from tqdm import tqdm
from utils import *
import numpy as np
category_list = ['common_tree_level2', 'common_flood damaged area_level2', 'common_flood debris damaged area_level2', 'common_street lamp_level2']

In [43]:
# 라벨링 json to txt
def convert_json(json_dir='./annotations_smartinside_segmentation/', use_segments=True, cls91to80=False): # json 파일들 경로
    save_dir = "./new_bbox_smartinside_segmentation/" # output directory 저장되는 경로

    # Import json
    for json_file in sorted(Path(json_dir).resolve().glob('*.json')): # json이 있는 폴더에서 모든 json파일에 대해 하나씩
        fn = Path(save_dir)  # folder name, 폴더 이름
        with open(json_file) as f: # 하나씩 json을 연다
            data = json.load(f) # data 변수에 json내용을 저장

        # Create image dict
        images = {'%g' % x['id']: x for x in data['images']}
        # Create image-annotations dict
        imgToAnns = defaultdict(list)
        for ann in data['annotations']:
            imgToAnns[ann['image_id']].append(ann)

        # Write labels file
        for img_id, anns in tqdm(imgToAnns.items(), desc=f'Annotations {json_file}'):
            img = images['%g' % img_id]
            h, w, f = img['height'], img['width'], img['file_name']

            bboxes = []
            segments = []
            for ann in anns:
                if ann['iscrowd']:
                    continue
                # The COCO box format is [top left x, top left y, width, height]
                box = np.array(ann['bbox'], dtype=np.float64)
                box[:2] += box[2:] / 2  # xy top-left corner to center
                box[[0, 2]] /= w  # normalize x
                box[[1, 3]] /= h  # normalize y
                if box[2] <= 0 or box[3] <= 0:  # if w <= 0 and h <= 0
                    continue

                cls = category_list.index(ann['category'])  # class
                box = [cls] + box.tolist()
                if box not in bboxes:
                    bboxes.append(box)
                # Segments
                if use_segments:
                    if len(ann['segmentation']) > 1:
                        s = merge_multi_segment(ann['segmentation'])
                        s = (np.concatenate(s, axis=0) / np.array([w, h])).reshape(-1).tolist()
                    else:
                        s = [j for i in ann['segmentation'] for j in i]  # all segments concatenated
                        s = (np.array(s).reshape(-1, 2) / np.array([w, h])).reshape(-1).tolist()
                    s = [cls] + s
                    if s not in segments:
                        segments.append(s)

            # Write
            with open((fn / f).with_suffix('.txt'), 'a') as file:
                for i in range(len(bboxes)):
                    line = *(segments[i] if use_segments else bboxes[i]),  # cls, box or segments
                    file.write(('%g ' * len(line)).rstrip() % line + '\n')


def min_index(arr1, arr2):
    """Find a pair of indexes with the shortest distance. 
    Args:
        arr1: (N, 2).
        arr2: (M, 2).
    Return:
        a pair of indexes(tuple).
    """
    dis = ((arr1[:, None, :] - arr2[None, :, :]) ** 2).sum(-1)
    return np.unravel_index(np.argmin(dis, axis=None), dis.shape)


In [ ]:
# img사이즈 640x640으로 변경하는 코드
import cv2
from pathlib import Path
import os

jpg_dir = "./dataset_hair_new_smartinside_segmentation/"
save_dir = "./dataset_resized_smartinside_segmentation(640x640)/"
dirs = os.listdir(jpg_dir)

for file in dirs:
    if os.path.isfile(jpg_dir+file):  #파일인지 아닌지 체크하기위함!
        img = cv2.imread(jpg_dir+file) # 이미지 읽기
        img_resized = cv2.resize(img, (640, 640)) #size 지정. 따로 변수를 빼는것도 나쁘지 않겠네요.
        cv2.imwrite(save_dir+file, img_resized)

In [45]:
# train test validation 으로 나누는 코드
import os
import random
import shutil

# 원본 파일과 라벨링 데이터의 경로
jpg_folder = "./dataset_resized_smartinside_segmentation(640x640)/"
txt_folder = "./new_bbox_smartinside_segmentation/"

# 분할된 데이터를 저장할 폴더 경로
train_img_folder = "./datasets/train/images"
train_txt_folder = "./datasets/train/labels"
test_img_folder = "./datasets/test/images"
test_txt_folder = "./datasets/test/labels"
valid_img_folder = "./datasets/valid/images"
valid_txt_folder = "./datasets/valid/labels"

# 분할 비율 (train:test:valid)
split_ratio = [0.7, 0.1, 0.2]

# 파일 목록 가져오기
jpg_files = os.listdir(jpg_folder)
txt_files = os.listdir(txt_folder)

# 파일 목록을 랜덤하게 섞음
random.shuffle(jpg_files)

# 파일 분할 인덱스 계산
num_files = len(jpg_files)
train_split = int(num_files * split_ratio[0])
test_split = int(num_files * (split_ratio[0] + split_ratio[1]))

# train 폴더로 파일 이동
for file in jpg_files[:train_split]:
    base_name = os.path.splitext(file)[0]
    txt_file = base_name + ".txt"
    if txt_file in txt_files:
        shutil.move(os.path.join(jpg_folder, file), os.path.join(train_img_folder, file))
        shutil.move(os.path.join(txt_folder, txt_file), os.path.join(train_txt_folder, txt_file))

# test 폴더로 파일 이동
for file in jpg_files[train_split:test_split]:
    base_name = os.path.splitext(file)[0]
    txt_file = base_name + ".txt"
    if txt_file in txt_files:
        shutil.move(os.path.join(jpg_folder, file), os.path.join(test_img_folder, file))
        shutil.move(os.path.join(txt_folder, txt_file), os.path.join(test_txt_folder, txt_file))

# valid 폴더로 파일 이동
for file in jpg_files[test_split:]:
    base_name = os.path.splitext(file)[0]
    txt_file = base_name + ".txt"
    if txt_file in txt_files:
        shutil.move(os.path.join(jpg_folder, file), os.path.join(valid_img_folder, file))
        shutil.move(os.path.join(txt_folder, txt_file), os.path.join(valid_txt_folder, txt_file))